In [15]:
# This file handles the classes for the valuation of a stock using traditional methods

import yfinance as yf
import pandas as pd
import numpy as np


# Valuation object to handle the traditional dcf valuation methods
class Valuation:
    def __init__(self, stock):
        self.stock = stock

    def get_value(self):
        raise NotImplementedError("Subclasses should implement this method")

    def test(self):
        raise NotImplementedError("Subclasses should implement this method")
    
    def get_valuation_ratios(self):
        raise NotImplementedError("Subclasses should implement this method")

"""
Valuation object to handle the traditional dcf valuation methods
Requires:
- stock object as yf.Ticker
- ten year treasury yield ideal holding point
- current inflation rate
- last years s and p 500 returns 
- dividend payout ratio for the company
"""
class DCFValuation(Valuation):
    def __init__(self, stock, inflation_rate, periods=4):
        super().__init__(stock)
        self.inflation_rate = inflation_rate
        self.periods = periods
        self.last_sp500_return = self.__get_market_rate_of_return()
        self.risk_free_rate = self.__get_risk_free_rate()

    def __get_treasury_yields(self):
        treasury_ticker = '^TNX'
        treasury_data = yf.Ticker(treasury_ticker)
        treasury_yield = treasury_data.history(period='5d')['Close'].iloc[-1]
        risk_free_rate = treasury_yield / 100
        
        return risk_free_rate

    def __get_market_rate_of_return(self):
        sp500 = yf.Ticker('^GSPC')
        sp500_data = sp500.history(period='10y')
        sp500_data['Yearly Return'] = sp500_data['Close'].pct_change(periods=252)
        yearly_returns = sp500_data['Yearly Return'].dropna()
        avg_annual_return = yearly_returns.mean()
        
        return avg_annual_return

    def __get_risk_free_rate(self):
        raw_risk_free_rate = self.__get_treasury_yields()
        nominal_risk_free_rate = (1 + raw_risk_free_rate) * (1 + self.inflation_rate) - 1

        return nominal_risk_free_rate

    def __get_cash_flows(self):
        try:
            cash_flows = self.stock.cash_flow.loc['Free Cash Flow'].iloc[:self.periods].to_numpy()
        except KeyError:
            cash_flows = self.stock.cash_flow.loc['Operating Cash Flow'].iloc[:self.periods].to_numpy()
        return list(cash_flows)
    
    def __get_growth_rate(self):
        cash_flows = self.__get_cash_flows()
        growth_rates = []
        for i in range(1, len(cash_flows)):
            yearOne, yearTwo = cash_flows[i - 1], cash_flows[i]
            curr_growth_rate = (yearTwo - yearOne) / yearOne
            growth_rates.append(curr_growth_rate)
        return (sum(growth_rates) / len(growth_rates))

    def cost_of_equity(self):
        # risk free rate + beta * (market rate of return - risk-free-rate of return)
        beta = self.stock.df.get('beta', 1)
        nominal_risk_free_rate = self.risk_free_rate
        market_rate_of_return = self.last_sp500_return
        cost_of_equity = (nominal_risk_free_rate + beta * (market_rate_of_return - nominal_risk_free_rate))

        return cost_of_equity

    def cost_of_debt(self):
        credit_spread = self.__get_credit_spread()
        tax_rate = self.__get_tax_rate()
        nominal_risk_free_rate = self.risk_free_rate    
        current_credit_spread = credit_spread[0]
        # (After tax cost of debt + Credit Spread) * (1 - tax rate)
        cost_of_debt = (nominal_risk_free_rate + current_credit_spread) * (1 - tax_rate)

        return cost_of_debt
    
    def get_wacc(self):
    
        tax_rate = self.__get_tax_rate()
        cost_of_equity = self.cost_of_equity()
        cost_of_debt = self.cost_of_debt()
        equity_weight, debt_weight = self.get_weights()
        # WACC = (equity weight * cost of equity) + debt weight * cosst of detb * (1 - tax rate)
        wacc = (equity_weight * cost_of_equity) + (debt_weight * cost_of_debt * (1 - tax_rate))

        return wacc

    def get_weights(self):
        def get_market_value_of_equity():
            market_cap = self.stock.df.get('marketCap')
            return market_cap
        
        def get_market_cap_of_debt():
            total_debt = self.stock.balance_sheet.loc['Total Debt'].iloc[0]
            return total_debt

        equity = get_market_value_of_equity()
        total_debt = get_market_cap_of_debt()

        equity_weight = equity / (equity + total_debt)
        debt_weight = total_debt / (equity + total_debt)

        return equity_weight, debt_weight
    
    def __get_tax_rate(self):
        return self.stock.income_stmt.loc['Tax Rate For Calcs'].iloc[0]
    
    def __get_credit_spread(self):
            real_risk_free_rate = self.risk_free_rate
            interest_expense = self.stock.obtain_interest_expense(periods=self.periods)
            total_debt = self.stock.balance_sheet.loc['Total Debt'].iloc[:self.periods].to_numpy()
            average_interest_expense = [self.risk_free_rate if interest_expense[i] == 0 else interest_expense[i] / total_debt[i] for i in range(len(total_debt))]
            credit_spread = list(map(lambda avg_interest: avg_interest - real_risk_free_rate, average_interest_expense))

            return credit_spread
    

    def __calcuate_terminal_value(self, growth_rate, wacc):
        # Infinite growth rate == 0.02
        final_year_fcf = self.stock.cash_flow.loc['Free Cash Flow'].iloc[0]
        if growth_rate < 0:
            perpetuity_growth_rate = 0
        else:
            perpetuity_growth_rate = 0.02
        # (final year fcf * (1 + perpetuity growth rate)) / (discount rate or wacc - perpetuiyt growth rate)
        terminal_value = (final_year_fcf * (1 + perpetuity_growth_rate)) / (wacc - perpetuity_growth_rate)

        return terminal_value

    def get_valuation_ratios(self):
        dcf_valuation_ratio = self.__get_dcf_value() / self.stock.df.get('marketCap')
        free_cash_flow = self.stock.cash_flow.loc['Free Cash Flow'].iloc[:self.periods].to_numpy()
        if len(free_cash_flow) < self.periods:
            free_cash_flow = np.append(free_cash_flow, free_cash_flow[-1] * self.periods - len(free_cash_flow))
        cash_flow1, cash_flow2, cash_flow3, cash_flow4 = free_cash_flow[0], free_cash_flow[1], free_cash_flow[2], free_cash_flow[3]
        growth_rate = self.__get_growth_rate()
        wacc = self.get_wacc()
        cost_of_equity = self.cost_of_equity()
        cost_of_debt = self.cost_of_debt()

        return [dcf_valuation_ratio, cash_flow1, cash_flow2, cash_flow3, cash_flow4, growth_rate, wacc, cost_of_equity, cost_of_debt]

    def __get_dcf_value(self):
        growth_rate = self.__get_growth_rate()
        wacc = self.get_wacc()
        cash_flows = self.__get_cash_flows()
        forecasted_fcfs = []

        # 1. Forecast the cash flows

        for i in range(self.periods):
            fcf = cash_flows[-1] * ((1 + growth_rate) ** (i + 1))
            forecasted_fcfs.append(fcf)

        # 2. Discount the cash flows

        discounted_fcfs = [fcf / ((1 + wacc) ** (i + 1)) for i, fcf in enumerate(forecasted_fcfs)]

        # 3. Calculate the terminal value

        terminal_value = self.__calcuate_terminal_value(growth_rate, wacc)

        # 4. Discount the terminal value

        discounted_terminal_value = terminal_value / ((1 + wacc) ** self.periods)

        # 5. Sum the present values

        dcf_value = sum(discounted_fcfs) + discounted_terminal_value

        return dcf_value

    def get_value(self):
        return self.__get_dcf_value()

    def test(self):
        try:
            market_cap = self.stock.df.get('marketCap')
            wacc = self.get_wacc()
            growth_rate = self.__get_growth_rate()
            terminal_value = self.__calcuate_terminal_value(growth_rate, wacc)
            dcf_value = self.__get_dcf_value()
            
            print(f"Ticker: {self.stock.ticker}")
            print(f"Risk Free Rate: {self.risk_free_rate}")
            print(f"Market Cap: {market_cap}")
            print(f"WACC: {wacc}")
            print(f"Growth Rate: {growth_rate}")
            print(f"Terminal Value: {terminal_value}")
            print(f"DCF Value: {dcf_value}")
            print(f"Valuation ratio: {dcf_value / market_cap:.2f}")
        except Exception as e:
            print(f"Failed to obtain data for {self.ticker}.")
            print(e)

"""
Object to handle the dividend discount model
We will assume a gordon growth model for the data and calculate the intrinsic value of the stock
"""
class DDMValuation(Valuation):
    def __init__(self, stock):
        super().__init__(stock)
        self.dividend_payout_ratio = self.__calculate_dividend_payout_ratio()

    def __calculate_dividend_payout_ratio(self):
        div_yield = self.stock.df.get('dividendYield', 0) * 100
        pe_ratio = self.stock.df.get('trailingPE', 0)
        dividend_payout_ratio = div_yield / (pe_ratio + 1e-10)
        if pd.isna(dividend_payout_ratio):
            dividend_payout_ratio = 0
        return dividend_payout_ratio
    
    def check(self):
        return self.dividend_payout_ratio > 0.15

    def __get_current_stock_price(self):
        return self.stock.df.get('previousClose')
    
    def __calculate_historical_growth(self, periods=0):
        if periods:
            pct_changes = self.stock.dividends.pct_change(periods=periods)
        else:
            pct_changes = self.stock.dividends.pct_change()
        lower, upper = pct_changes.quantile(0.025), pct_changes.quantile(0.975)
        filtered_pct_changes = pct_changes[(pct_changes >= lower) & (pct_changes <= upper)]
        return filtered_pct_changes.mean()

    def __calculate_rolling_average_growth(self, window=10):
        rolling_growth = self.stock.dividends.pct_change().rolling(window=window).mean()
        return rolling_growth.mean()

    def __calculate_combined_growth(self, short_period=5, long_period=10):
        short_term_growth = self.__calculate_historical_growth(periods=short_period)
        long_term_growth = self.__calculate_historical_growth(periods=long_period)
        combined_growth = (0.4 * short_term_growth) + (0.6 * long_term_growth)
        return combined_growth

    def __get_realistic_growth_rate(self):
        historical_growth = self.__calculate_historical_growth()
        rolling_average_growth = self.__calculate_rolling_average_growth()
        combined_growth = self.__calculate_combined_growth()
        cost_of_equity = self.cost_of_equity()
        realistic_growth_rate = max(min(historical_growth, rolling_average_growth, combined_growth), 0.02)
        optimal_growth_rate = min(max(historical_growth, rolling_average_growth, combined_growth), cost_of_equity * 0.9)
        return {
            'historical_growth': historical_growth,
            'rolling_average_growth': rolling_average_growth,
            'combined_growth': combined_growth,
            'realistic_growth_rate': realistic_growth_rate,
            'optimal_growth_rate': optimal_growth_rate
        }


    def get_value(self):
        if not self.check():
            print("Dividend Payout Ratio is too low to calculate DDM value.")
            return np.nan
        return self.__get_ddm_value()
    
    def cost_of_equity(self):
        dcf = DCFValuation(self.stock, 0.02)
        return dcf.cost_of_equity()
    
    def __get_current_dividend(self):
        dividends = self.stock.dividends
        current_dividend = dividends.iloc[-1]

        return current_dividend

    def __get_ddm_value(self):
        current_dividend = self.__get_current_dividend()
        if pd.isna(current_dividend):
            return np.nan

        cost_of_equity = self.cost_of_equity()
        expected_growth_rate = self.__get_realistic_growth_rate().get('realistic_growth_rate')
        optimal_growth_rate = self.__get_realistic_growth_rate().get('optimal_growth_rate')
        if pd.isna(expected_growth_rate) or cost_of_equity <= expected_growth_rate:
            expected_growth_rate = 0.8 * cost_of_equity
        expected_dividend_next_year = current_dividend * (1 + expected_growth_rate)

        intrinsic_value_min = expected_dividend_next_year / (cost_of_equity - expected_growth_rate)
        intrinsic_value_max = expected_dividend_next_year / (cost_of_equity - optimal_growth_rate)
        return [intrinsic_value_min, intrinsic_value_max]

    def get_valuation_ratios(self):
        ddm_valuation_ratio_min = self.__get_ddm_value()[0] / self.stock.df.get('marketCap')
        ddm_valuation_ratio_max = self.__get_ddm_value()[1] / self.stock.df.get('marketCap')    
        expected_growth_rate = self.__get_realistic_growth_rate().get('realistic_growth_rate')
        
        return [ddm_valuation_ratio_min, ddm_valuation_ratio_max, expected_growth_rate]

    def test(self):
        try:
            current_stock_price = self.__get_current_stock_price()
            expected_growth_rate = self.__get_realistic_growth_rate().get('realistic_growth_rate')
            required_rate_of_return = self.cost_of_equity()
            current_dividend = self.__get_current_dividend()
            intrinsic_value_min, intrinsic_value_max = self.__get_ddm_value()[0], self.__get_ddm_value()[1]
            print(f"Ticker: {self.stock.ticker}")
            print(f"Current Dividend: {current_dividend}")
            print(f"Current Stock Price: {current_stock_price}")
            print(f"Expected Growth Rate: {expected_growth_rate:.4f}")
            print(f"Required Rate of Return: {required_rate_of_return:.4f}")
            print(f"Intrinsic Value min: {intrinsic_value_min:.2f}")
            print(f"Intrinsic Value max: {intrinsic_value_max:.2f}")
            print(f"Dividend Payout Ratio: {self.dividend_payout_ratio:.2f}")
        except Exception as e:
            print(f"Failed to obtain data for {self.ticker}.")
            print(e)

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np


# Valuation object to handle the traditional dcf valuation methods
class Valuation:
    def __init__(self, stock):
        self.stock = stock

    def get_value(self):
        raise NotImplementedError("Subclasses should implement this method")

    def test(self):
        raise NotImplementedError("Subclasses should implement this method")
    
    def get_valuation_ratios(self):
        raise NotImplementedError("Subclasses should implement this method")


class DCFValuation(Valuation):
    def __init__(self, stock, inflation_rate, periods=4):
        super().__init__(stock)
        self.inflation_rate = inflation_rate
        self.periods = periods
        self.last_sp500_return = self.__get_market_rate_of_return()
        self.risk_free_rate = self.__get_risk_free_rate()

    def __get_treasury_yields(self):
        treasury_ticker = '^TNX'
        treasury_data = yf.Ticker(treasury_ticker)
        treasury_yield = treasury_data.history(period='5d')['Close'].iloc[-1]
        risk_free_rate = treasury_yield / 100
        return risk_free_rate

    def __get_market_rate_of_return(self):
        sp500 = yf.Ticker('^GSPC')
        sp500_data = sp500.history(period='10y')
        sp500_data['Yearly Return'] = sp500_data['Close'].pct_change(periods=252)
        yearly_returns = sp500_data['Yearly Return'].dropna()
        avg_annual_return = yearly_returns.mean()
        return avg_annual_return

    def __get_risk_free_rate(self):
        raw_risk_free_rate = self.__get_treasury_yields()
        nominal_risk_free_rate = (1 + raw_risk_free_rate) * (1 + self.inflation_rate) - 1
        return nominal_risk_free_rate

    def __get_cash_flows(self):
        try:
            cash_flows = self.stock.cash_flow.loc['Free Cash Flow'].iloc[:self.periods].to_numpy()
        except KeyError:
            cash_flows = self.stock.cash_flow.loc['Operating Cash Flow'].iloc[:self.periods].to_numpy()
        
        # Ensure cash flows are non-negative
        cash_flows = np.where(cash_flows < 0, 0, cash_flows)
        return list(cash_flows)

    def __get_growth_rate(self):
        cash_flows = self.__get_cash_flows()
        growth_rates = []
        for i in range(1, len(cash_flows)):
            yearOne, yearTwo = cash_flows[i - 1], cash_flows[i]
            if yearOne <= 0 or yearTwo <= 0:
                continue
            curr_growth_rate = (yearTwo - yearOne) / yearOne
            growth_rates.append(curr_growth_rate)
        if not growth_rates:
            return 0
        avg_growth_rate = sum(growth_rates) / len(growth_rates)
        # We cap at 20% (to be prudent)
        return min(max(avg_growth_rate, 0), 0.20)



    def cost_of_equity(self):
        beta = self.stock.df.get('beta', 1)
        nominal_risk_free_rate = self.risk_free_rate
        market_rate_of_return = self.last_sp500_return
        cost_of_equity = nominal_risk_free_rate + beta * (market_rate_of_return - nominal_risk_free_rate)
        return cost_of_equity

    def cost_of_debt(self):
        credit_spread = self.__get_credit_spread()
        tax_rate = self.__get_tax_rate()
        nominal_risk_free_rate = self.risk_free_rate    
        current_credit_spread = credit_spread[0]
        cost_of_debt = (nominal_risk_free_rate + current_credit_spread) * (1 - tax_rate)
        return cost_of_debt
    
    def get_wacc(self):
        tax_rate = self.__get_tax_rate()
        cost_of_equity = self.cost_of_equity()
        cost_of_debt = self.cost_of_debt()
        equity_weight, debt_weight = self.get_weights()
        wacc = (equity_weight * cost_of_equity) + (debt_weight * cost_of_debt * (1 - tax_rate))
        return max(wacc, 0)

    def get_weights(self):
        def get_market_value_of_equity():
            market_cap = self.stock.df.get('marketCap')
            return market_cap
        
        def get_market_cap_of_debt():
            total_debt = self.stock.balance_sheet.loc['Total Debt'].iloc[0]
            return total_debt

        equity = get_market_value_of_equity()
        total_debt = get_market_cap_of_debt()
        equity_weight = equity / (equity + total_debt)
        debt_weight = total_debt / (equity + total_debt)
        return equity_weight, debt_weight
    
    def __get_tax_rate(self):
        return self.stock.income_stmt.loc['Tax Rate For Calcs'].iloc[0]
    
    def __get_credit_spread(self):
        real_risk_free_rate = self.risk_free_rate
        interest_expense = self.stock.obtain_interest_expense(periods=self.periods)
        total_debt = self.stock.balance_sheet.loc['Total Debt'].iloc[:self.periods].to_numpy()
        average_interest_expense = [self.risk_free_rate if interest_expense[i] == 0 else interest_expense[i] / total_debt[i] for i in range(len(total_debt))]
        credit_spread = [avg_interest - real_risk_free_rate for avg_interest in average_interest_expense]
        return credit_spread

    def __calculate_terminal_value(self, growth_rate, wacc):
        final_year_fcf = self.stock.cash_flow.loc['Free Cash Flow'].iloc[0]
        perpetuity_growth_rate = 0 if growth_rate < 0 else 0.02
        if wacc <= perpetuity_growth_rate:
            wacc = perpetuity_growth_rate + 0.01
        terminal_value = (final_year_fcf * (1 + perpetuity_growth_rate)) / (wacc - perpetuity_growth_rate)
        return terminal_value

    def get_valuation_ratios(self):
        dcf_valuation_ratio = self.__get_dcf_value() / self.stock.df.get('marketCap')
        free_cash_flow = self.stock.cash_flow.loc['Free Cash Flow'].iloc[:self.periods].to_numpy()
        if len(free_cash_flow) < self.periods:
            free_cash_flow = np.append(free_cash_flow, free_cash_flow[-1] * self.periods - len(free_cash_flow))
        cash_flows = free_cash_flow[:4]
        growth_rate = self.__get_growth_rate()
        wacc = self.get_wacc()
        cost_of_equity = self.cost_of_equity()
        cost_of_debt = self.cost_of_debt()
        return [dcf_valuation_ratio, *cash_flows, growth_rate, wacc, cost_of_equity, cost_of_debt]

    def __get_dcf_value(self):
        growth_rate = self.__get_growth_rate()
        wacc = self.get_wacc()
        cash_flows = self.__get_cash_flows()
        forecasted_fcfs = [cash_flows[-1] * ((1 + growth_rate) ** (i + 1)) for i in range(self.periods)]
        discounted_fcfs = [fcf / ((1 + wacc) ** (i + 1)) for i, fcf in enumerate(forecasted_fcfs)]
        terminal_value = self.__calculate_terminal_value(growth_rate, wacc)
        discounted_terminal_value = terminal_value / ((1 + wacc) ** self.periods)
        dcf_value = sum(discounted_fcfs) + discounted_terminal_value

        print(f"Growth Rate: {growth_rate}")
        print(f"WACC: {wacc}")
        print(f"Cash Flows: {cash_flows}")
        print(f"Forecasted FCFs: {forecasted_fcfs}")
        print(f"Discounted FCFs: {discounted_fcfs}")
        print(f"Terminal Value: {terminal_value}")
        print(f"Discounted Terminal Value: {discounted_terminal_value}")
        print(f"DCF Value: {dcf_value}")
        return dcf_value

    def get_value(self):
        return self.__get_dcf_value()

    def test(self):
        try:
            market_cap = self.stock.df.get('marketCap')
            wacc = self.get_wacc()
            growth_rate = self.__get_growth_rate()
            terminal_value = self.__calculate_terminal_value(growth_rate, wacc)
            dcf_value = self.__get_dcf_value()
            print(f"Ticker: {self.stock.ticker}")
            print(f"Risk Free Rate: {self.risk_free_rate}")
            print(f"Market Cap: {market_cap}")
            print(f"WACC: {wacc}")
            print(f"Growth Rate: {growth_rate}")
            print(f"Terminal Value: {terminal_value}")
            print(f"DCF Value: {dcf_value}")
            print(f"Valuation ratio: {dcf_value / market_cap:.2f}")
        except Exception as e:
            print(f"Failed to obtain data for {self.ticker}.")
            print(e)


class DDMValuation(Valuation):
    def __init__(self, stock):
        super().__init__(stock)
        self.dividend_payout_ratio = self.__calculate_dividend_payout_ratio()

    def __calculate_dividend_payout_ratio(self):
        div_yield = self.stock.df.get('dividendYield', 0) * 100
        pe_ratio = self.stock.df.get('trailingPE', 0)
        dividend_payout_ratio = div_yield / (pe_ratio + 1e-10)
        return 0 if pd.isna(dividend_payout_ratio) else dividend_payout_ratio
    
    def check(self):
        return self.dividend_payout_ratio > 0.15

    def __get_current_stock_price(self):
        return self.stock.df.get('previousClose')
    
    def __calculate_historical_growth(self, periods=0):
        pct_changes = self.stock.dividends.pct_change(periods=periods) if periods else self.stock.dividends.pct_change()
        lower, upper = pct_changes.quantile(0.025), pct_changes.quantile(0.975)
        filtered_pct_changes = pct_changes[(pct_changes >= lower) & (pct_changes <= upper)]
        return filtered_pct_changes.mean()

    def __calculate_rolling_average_growth(self, window=10):
        rolling_growth = self.stock.dividends.pct_change().rolling(window=window).mean()
        return rolling_growth.mean()

    def __calculate_combined_growth(self, short_period=5, long_period=10):
        short_term_growth = self.__calculate_historical_growth(periods=short_period)
        long_term_growth = self.__calculate_historical_growth(periods=long_period)
        combined_growth = (0.4 * short_term_growth) + (0.6 * long_term_growth)
        return combined_growth

    def __get_realistic_growth_rate(self):
        historical_growth = self.__calculate_historical_growth()
        rolling_average_growth = self.__calculate_rolling_average_growth()
        combined_growth = self.__calculate_combined_growth()
        cost_of_equity = self.cost_of_equity()
        realistic_growth_rate = max(min(historical_growth, rolling_average_growth, combined_growth), 0.02)
        optimal_growth_rate = min(max(historical_growth, rolling_average_growth, combined_growth), cost_of_equity * 0.9)
        return {
            'historical_growth': historical_growth,
            'rolling_average_growth': rolling_average_growth,
            'combined_growth': combined_growth,
            'realistic_growth_rate': realistic_growth_rate,
            'optimal_growth_rate': optimal_growth_rate
        }

    def get_value(self):
        if not self.check():
            print("Dividend Payout Ratio is too low to calculate DDM value.")
            return np.nan
        return self.__get_ddm_value()
    
    def cost_of_equity(self):
        dcf = DCFValuation(self.stock, 0.02)
        return dcf.cost_of_equity()
    
    def __get_current_dividend(self):
        dividends = self.stock.dividends
        current_dividend = dividends.iloc[-1]
        return current_dividend

    def __get_ddm_value(self):
        current_dividend = self.__get_current_dividend()
        if pd.isna(current_dividend):
            return np.nan

        cost_of_equity = self.cost_of_equity()
        growth_data = self.__get_realistic_growth_rate()
        expected_growth_rate = growth_data.get('realistic_growth_rate')
        optimal_growth_rate = growth_data.get('optimal_growth_rate')

        if pd.isna(expected_growth_rate) or cost_of_equity <= expected_growth_rate:
            expected_growth_rate = 0.8 * cost_of_equity

        expected_dividend_next_year = current_dividend * (1 + expected_growth_rate)
        intrinsic_value_min = expected_dividend_next_year / (cost_of_equity - expected_growth_rate)
        intrinsic_value_max = expected_dividend_next_year / (cost_of_equity - optimal_growth_rate)
        return [intrinsic_value_min, intrinsic_value_max]

    def get_valuation_ratios(self):
        ddm_values = self.__get_ddm_value()
        ddm_valuation_ratio_min = ddm_values[0] / self.stock.df.get('marketCap')
        ddm_valuation_ratio_max = ddm_values[1] / self.stock.df.get('marketCap')    
        expected_growth_rate = self.__get_realistic_growth_rate().get('realistic_growth_rate')
        return [ddm_valuation_ratio_min, ddm_valuation_ratio_max, expected_growth_rate]

    def test(self):
        try:
            current_stock_price = self.__get_current_stock_price()
            growth_data = self.__get_realistic_growth_rate()
            expected_growth_rate = growth_data.get('realistic_growth_rate')
            required_rate_of_return = self.cost_of_equity()
            current_dividend = self.__get_current_dividend()
            intrinsic_value_min, intrinsic_value_max = self.__get_ddm_value()
            print(f"Ticker: {self.stock.ticker}")
            print(f"Current Dividend: {current_dividend}")
            print(f"Current Stock Price: {current_stock_price}")
            print(f"Expected Growth Rate: {expected_growth_rate:.4f}")
            print(f"Required Rate of Return: {required_rate_of_return:.4f}")
            print(f"Intrinsic Value min: {intrinsic_value_min:.2f}")
            print(f"Intrinsic Value max: {intrinsic_value_max:.2f}")
            print(f"Dividend Payout Ratio: {self.dividend_payout_ratio:.2f}")
        except Exception as e:
            print(f"Failed to obtain data for {self.stock.ticker}.")
            print(e)


In [21]:
import yfinance as yf
import pandas as pd
import numpy as np

class Stock:
    def __init__(self, ticker, inflation_rate=0.0160, years=1, isQuarter=False):
        self.ticker = ticker
        self.years = years
        self.isQuarter = isQuarter
        self.inflation_rate = inflation_rate
        self.stock = yf.Ticker(ticker)
        self.df = self.stock.info
        if self.df is None:
            raise ValueError(f"Failed to get data for {self.ticker}.")
        self.income_stmt = self.stock.financials
        self.balance_sheet = self.stock.balance_sheet
        self.cash_flow = self.stock.cashflow
        if self.income_stmt.empty or self.balance_sheet.empty or self.cash_flow.empty:
            raise ValueError(f"Failed to get financial data for {self.ticker}.")
        self.dividends = self.stock.dividends
        self.dcf_valuation = DCFValuation(self, self.inflation_rate)
        self.ddm_valuation = DDMValuation(self)
        self.market_cap = self.df.get('marketCap')

        self.validate_data()

    def validate_data(self):
        if not self.ticker or not isinstance(self.ticker, str):
            raise ValueError("Ticker must be a non-empty string.")
        if not isinstance(self.inflation_rate, (int, float)):
            raise ValueError("Inflation rate must be a number.")

    def get_history(self, period="max"):
        try:
            return self.stock.history(period=period)
        except Exception as e:
            print(f"Failed to get history for {self.ticker}: {e}")
            return pd.DataFrame()

    def refresh_data(self):
        try:
            self.stock = yf.Ticker(self.ticker)
            self.df = self.stock.info
            self.income_stmt = self.stock.financials
            self.balance_sheet = self.stock.balance_sheet
            self.cash_flow = self.stock.cashflow
            self.dividends = self.stock.dividends
            print(f"Data refreshed for {self.ticker}.")
        except Exception as e:
            print(f"Failed to refresh data for {self.ticker}: {e}")

    def get_value(self, attribute, default=None, isBig=False):
        return self.df.get(attribute, default) if not isBig else self.df.get(attribute, default) / 100

    def test(self, fn):
        try:
            data = fn()
            print(data)
        except Exception as e:
            print(f"Failed to execute function for {self.ticker}: {e}")

    # Util methods

    def _get_metric_from_df(self, key, default=0):
        return self.df.get(key, default)

    def _get_metric_from_income_stmt(self, key, default=0):
        try:
            return self.income_stmt.loc[key].iloc[0] / 100
        except KeyError:
            return default

    def _get_metric_from_balance_sheet(self, key, default=0):
        try:
            return self.balance_sheet.loc[key].iloc[0] / 100
        except KeyError:
            return default

    def _get_metric_from_cashflow(self, key, default=0):
        try:
            return self.cash_flow.loc[key].iloc[0] / 100
        except KeyError:
            return default

    def _get_sum_of_keys(self, primary_key, keys):
        try:
            return self.balance_sheet.loc[primary_key].iloc[0]
        except KeyError:
            total = 0
            for key in keys:
                if key in self.balance_sheet.index:
                    total += self.balance_sheet.loc[key].sum()
                else:
                    print(f"Key '{key}' not found in balance sheet.")
            return total

    def _get_ratio(self, numerator, denominator, default=0):
        return numerator / denominator if denominator != 0 else default

    def __str__(self):
        return f"Stock({self.ticker})"


class StockData(Stock):
    def __init__(self, ticker, inflation_rate=0.0160):
        super().__init__(ticker, inflation_rate)

    def build_data(self):
        # Basic financial metrics
        financial_metrics = {
            'Ticker': self.ticker,
            'Market Cap': self.market_cap,
            'Revenue': self.df.get('totalRevenue', 0),
            'Sector': self.df.get('sector', 'Unknown'),
            'Free Cash Flow': self.df.get('freeCashflow', 0),
            'Capital Expenditure': abs(self.__get_cap_ex()),
            'Net Income': self.df.get('netIncomeToCommon', 0),
            'Operating Cash Flow': self._get_metric_from_cashflow('Operating Cash Flow', 0),
            'EBITDA': self.df.get('ebitda', 0)
        }

        current_assets, total_assets = self.__get_assets()
        interest_expense = self.obtain_interest_expense()

        valuation_metrics = self.__get_valuation_metrics(financial_metrics, current_assets, total_assets, interest_expense)
        quality_metrics = self.__get_quality_metrics(financial_metrics, current_assets, total_assets, interest_expense)
        economic_moat_metrics = self.__get_economic_moat_metrics(financial_metrics)
        price_data = self.__get_price_data()

        value_data = {**financial_metrics, **valuation_metrics, **quality_metrics, **economic_moat_metrics, **price_data}

        # Convert to DataFrame
        value_data_df = pd.DataFrame([value_data])

        # List of columns to be divided by 100
        cols_to_divide = ['Market Cap', 'Revenue', 'Free Cash Flow', 'Capital Expenditure', 'Net Income', 'Operating Cash Flow', 'EBITDA', 'DCF Value Billions', 'Operating Cash Flow Billions', 'Retained Earnings Billions', 'Economic Value Added Billions', 'Earnings Power Value']

        # Divide the selected columns by 100
        for col in cols_to_divide:
            if col in value_data_df.columns:
                value_data_df[col] = value_data_df[col] / 1e9

        return value_data_df

    def __get_total_assets(self):
        return self._get_sum_of_keys('Total Assets', [
            'Cash Cash Equivalents And Short Term Investments',
            'Other Short Term Investments',
            'Cash And Cash Equivalents',
            'Cash Equivalents',
            'Cash Financial',
            'Receivables',
            'Other Receivables',
            'Accounts Receivable',
            'Inventory',
            'Other Current Assets',
            'Net PPE',
            'Investments And Advances',
            'Long Term Equity Investment',
            'Investments In Other Ventures Under Equity Method',
            'Goodwill And Other Intangible Assets',
            'Other Intangible Assets',
            'Goodwill',
            'Properties',
            'Machinery Furniture Equipment',
            'Land And Improvements'
        ])

    def __get_current_assets(self):
        return self._get_sum_of_keys('Current Assets', [
            'Cash Cash Equivalents And Short Term Investments',
            'Other Short Term Investments',
            'Cash And Cash Equivalents',
            'Cash Equivalents',
            'Cash Financial',
            'Receivables',
            'Other Receivables',
            'Accounts Receivable',
            'Inventory',
            'Other Current Assets'
        ])

    def __get_assets(self):
        total_assets = self.__get_total_assets()
        current_assets = self.__get_current_assets()
        return current_assets, total_assets

    def __get_valuation_metrics(self, financial_metrics, current_assets, total_assets, interest_expense):
        dcf_value = self.__get_dcf_value()
        if not self.ddm_valuation.check():
            ddm_valuation_ratio_min, ddm_valuation_ratio_max, dividend_expected_growth_rate = 0, 0, 0
        else:
            ddm_valuation_data = self.__get_ddm_values()
            ddm_valuation_ratio_min, ddm_valuation_ratio_max, dividend_expected_growth_rate = ddm_valuation_data

        dcf_valuation_data = self.__get_dcf_values()
        dcf_valuation_ratio, cash_flow1, cash_flow2, cash_flow3, cash_flow4, growth_rate, wacc, cost_of_equity, cost_of_debt = dcf_valuation_data
        strong_buy, buy, hold, sell, strong_sell = self.__parse_analyst_reccomendations()
        equity_weight, debt_weight = self.dcf_valuation.get_weights()

        pe_ratio = self._get_metric_from_df('trailingPE', 0)
        div_yield = self._get_metric_from_df('dividendYield', 0) * 100
        eps_growth = self.__get_eps_growth()
        return {
            'DCF Value Billions': dcf_value,
            'DCF Valuation Ratio': dcf_valuation_ratio,
            'DDM Valuation Ratio Min': ddm_valuation_ratio_min,
            'DDM Valuation Ratio Max': ddm_valuation_ratio_max,
            'Dividend Expected Growth Rate': dividend_expected_growth_rate,
            'P/E Ratio': pe_ratio,
            'P/B Ratio': self._get_metric_from_df('priceToBook', 0),
            'P/S Ratio': self._get_metric_from_df('priceToSalesTrailing12Months', 0),
            'P/FCF Ratio': self._get_ratio(self.market_cap, financial_metrics['Free Cash Flow']),
            'Dividend Yield': div_yield,
            'EPS': self._get_metric_from_df('trailingEps', 0),
            'EPS Growth': eps_growth,
            'Revenue Growth': self.__get_revenue_growth(),
            'PEG Ratio': self._get_ratio(pe_ratio, eps_growth),
            'Cash Flow 1 Billions': cash_flow1,
            'Cash Flow 2 Billions': cash_flow2,
            'Cash Flow 3 Billions': cash_flow3,
            'Cash Flow 4 Billions': cash_flow4,
            'Growth Rate': growth_rate,
            "NCAVPS": self.__get_net_current_asset_value_per_share(),
            'WACC': wacc,
            'Equity Weight': equity_weight,
            'Debt Weight': debt_weight,
            'Cost of Equity': cost_of_equity,
            'Cost of Debt': cost_of_debt,
            'Enterprise to Revenue': self._get_metric_from_df('enterpriseToRevenue', 0),
            'Enterprise to EBITDA': self._get_metric_from_df('enterpriseToEbitda', 0),
            'Total Cash Per Share': self._get_metric_from_df('totalCashPerShare', 0),
            'Interest Expense Billions': interest_expense / 100,
            'Strong Buy': strong_buy,
            'Buy': buy,
            'Hold': hold,
            'Sell': sell,
            'Strong Sell': strong_sell
        }

    def __get_quality_metrics(self, financial_metrics, current_assets, total_assets, interest_expense):
        current_liabilities = self.__get_current_liabilities()
        net_income = financial_metrics['Net Income']
        roe = self.__get_return_on_equity()
        de_ratio = self._get_metric_from_df('debtToEquity', 0)

        return {
            'Net Margin': self._get_ratio(net_income, financial_metrics['Revenue']),
            'Gross Margin': self.__get_gross_margin(),
            'ROA': self._get_metric_from_df('returnOnAssets', 0),
            'ROE': roe,
            'Current Ratio': self._get_ratio(current_assets, current_liabilities),
            'Quick Ratio': self.__get_quick_ratio(),
            'D/E Ratio': de_ratio,
            'Dividend Payout Ratio': self.__calculate_dividend_payout_ratio(),
            'FCF Yield': self._get_ratio(financial_metrics['Free Cash Flow'], self.market_cap) * 100,
            'Capex_to_sales': self._get_ratio(abs(financial_metrics['Capital Expenditure']), financial_metrics['Revenue']),
            'Operating Margin': self.__get_operating_margin(),
            'Debt to Asset Ratio': self._get_ratio(de_ratio, 1 + de_ratio),
            'Equity to Asset Ratio': self.__get_debt_to_asset(),
            'Interest Coverage Ratio': self._get_ratio(financial_metrics['Revenue'], interest_expense),
            'NCAVPS': self.__get_net_current_asset_value_per_share(),
            'ROIC': self.__get_roic(),
            'Altman Z Score': self.__get_altman_z_score(),
            'Operating Cash Flow Billions': financial_metrics['Operating Cash Flow'],
            'Asset Turnover Ratio': self.__get_turnover_ratios()[0],
            'Inventory Turnover Ratio': self.__get_turnover_ratios()[1],
            'Retained Earnings Billions': self.__get_retained_earnings(),
            'Earnings Power Value': self.__get_earnings_power_value()
        }

    def __get_price_data(self):
        current_price = self.get_history(period="1d")['Close'].iloc[-1]
        historical_data = self.get_history(period="1y")

        moving_average_50 = historical_data['Close'].rolling(window=50).mean().iloc[-1]
        moving_average_200 = historical_data['Close'].rolling(window=200).mean().iloc[-1]

        year_high = historical_data['Close'].max()
        year_low = historical_data['Close'].min()

        average_volume_50 = historical_data['Volume'].rolling(window=50).mean().iloc[-1]

        day_high = self.get_history(period="1d")['High'].iloc[-1]
        day_low = self.get_history(period="1d")['Low'].iloc[-1]

        price_data = {
            'Current Price': current_price,
            '50-Day Moving Average': moving_average_50,
            '200-Day Moving Average': moving_average_200,
            '52-Week High': year_high,
            '52-Week Low': year_low,
            'Average Volume (50 days)': average_volume_50,
            'Day High': day_high,
            'Day Low': day_low,
            'Previous Close': historical_data['Close'].iloc[-2],
            'Open': self.get_history(period='1d')['Open'].iloc[-1]
        }
        
        return price_data

    def __get_economic_moat_metrics(self, financial_metrics):
        free_cash_flow = financial_metrics['Free Cash Flow']
        revenue = financial_metrics['Revenue']
        fcf_ratio = self._get_ratio(free_cash_flow, revenue)
        eva = self.__get_eva()
        sgr = self.__get_sgr()

        return {
            'Economic Value Added Billions': eva,
            'FCF Ratio': fcf_ratio,
            'Sustainable Growth Rate': sgr
        }

    def __get_quick_ratio(self):
        quick_ratio = self._get_metric_from_df('quickRatio', 0)
        if quick_ratio == 0:
            inventory = self._get_metric_from_balance_sheet('Inventory', 0)
            current_assets = self.__get_current_assets()
            current_liabilities = self.__get_current_liabilities()
            quick_ratio = self._get_ratio(current_assets - inventory, current_liabilities)
        return quick_ratio
    
    def __get_debt_to_asset(self):
        debt = self._get_metric_from_balance_sheet('Total Debt', 0)
        assets = self._get_metric_from_balance_sheet('Total Assets', 0)
        return self._get_ratio(debt, assets)

    def __get_return_on_equity(self):
        roe = self._get_metric_from_df('returnOnEquity', 0)
        if roe == 0:
            net_income = self._get_metric_from_df('netIncomeToCommon', 0)
            equity = self._get_metric_from_balance_sheet('Total Equity', 0)
            roe = self._get_ratio(net_income, equity)
        return roe

    def __get_gross_margin(self):
        gross_margin = self._get_metric_from_df('grossMargins', 0)
        if gross_margin == 0:
            gross_profit = self._get_metric_from_income_stmt('Gross Profit', 0)
            revenue = self._get_metric_from_df('totalRevenue', 0)
            gross_margin = self._get_ratio(gross_profit, revenue)
        return gross_margin

    def __get_eps_growth(self):
        eps_growth = self._get_metric_from_df('earningsQuarterlyGrowth', 0)
        if eps_growth == 0:
            try:
                eps_diluted_array = self.income_stmt.loc['Diluted EPS'].to_numpy()[:4]
                eps_growth = self._get_ratio(eps_diluted_array[0] - eps_diluted_array[1], eps_diluted_array[1])
            except KeyError:
                return 0
        return eps_growth

    def __get_revenue_growth(self):
        revenue_growth = self._get_metric_from_df('revenueQuarterlyGrowth', 0)
        if revenue_growth == 0:
            try:
                revenue_array = self.income_stmt.loc['Total Revenue'].to_numpy()[:4]
                revenue_growth = self._get_ratio(revenue_array[0] - revenue_array[1], revenue_array[1])
            except KeyError:
                return 0
        return revenue_growth

    def __get_operating_margin(self):
        operating_margin = self._get_metric_from_df('operatingMargins', 0)
        if operating_margin == 0:
            operating_income = self._get_metric_from_income_stmt('Operating Income', 0)
            revenue = self._get_metric_from_df('totalRevenue', 0)
            operating_margin = self._get_ratio(operating_income, revenue)
        return operating_margin

    def __get_roic(self):
        net_income = self._get_metric_from_df('netIncomeToCommon', 0)
        total_assets = self.__get_total_assets()
        current_liabilities = self.__get_current_liabilities()
        return self._get_ratio(net_income, total_assets - current_liabilities)

    def __get_ebit(self):
        ebit = self._get_metric_from_income_stmt('EBIT', 0)
        if ebit == 0:
            net_income = self._get_metric_from_df('netIncomeToCommon', 0)
            special_income_charges = self._get_metric_from_income_stmt('Special Income Charges', 0)
            tax_provision = self._get_metric_from_income_stmt('Tax Provision', 0)
            interest_expense = self.obtain_interest_expense()
            ebit = net_income + interest_expense + tax_provision + special_income_charges
        return ebit

    def __get_earnings_power_value(self):
        ebit = self.__get_ebit()
        tax_rate = self._get_metric_from_income_stmt('Tax Rate For Calcs', 0)
        wacc = self.dcf_valuation.get_wacc()
        epv = self._get_ratio(ebit * (1 - tax_rate), wacc)
        if np.isnan(epv):
            epv = 0
        return epv

    def __get_altman_z_score(self):
        ebit = self.__get_ebit()
        working_capital = self.__get_current_assets() - self.__get_current_liabilities()
        retained_earnings = self.__get_retained_earnings()
        total_assets = self.__get_total_assets()
        total_liabilities = self.__get_total_liabilities()
        sales = self._get_metric_from_df('totalRevenue', 0)
        z = (1.2 * self._get_ratio(working_capital, total_assets) +
             1.4 * self._get_ratio(retained_earnings, total_assets) +
             3.3 * self._get_ratio(ebit, total_assets) +
             0.6 * self._get_ratio(self.market_cap, total_liabilities) +
             1.0 * self._get_ratio(sales, total_assets))
        return z

    def __get_turnover_ratios(self):
        sales = self._get_metric_from_df('totalRevenue', 0)
        total_assets = self.__get_total_assets()
        inventory = self._get_metric_from_balance_sheet('Inventory', 0)
        return self._get_ratio(sales, total_assets), self._get_ratio(sales, inventory)

    def _get_dividend_coverage(self):
        eps = self._get_metric_from_df('trailingEps', 0)
        dps = self._get_metric_from_df('dividendRate', 0)
        return self._get_ratio(eps, dps) if dps else np.nan

    def obtain_interest_expense(self, periods=0):
        interest_keys = ['Interest Expense Non Operating', 'Interest Expense', 'Net Non Operating Interest Income Expense']
        for key in interest_keys:
            if key in self.income_stmt.index:
                if periods > 0:
                    try:
                        interest_expense = self.income_stmt.loc[key].iloc[:periods].to_numpy()
                    except IndexError:
                        print(f"Not enough data for {periods} periods in key '{key}'. Returning available data.")
                        interest_expense = self.income_stmt.loc[key].to_numpy()
                else:
                    interest_expense = self.income_stmt.loc[key].iloc[0]
                return interest_expense
        if periods > 0:
            return np.zeros(periods)
        return 0

    def __get_eva(self):
        net_income = self._get_metric_from_income_stmt('Net Income', 0)
        tax_rate = self._get_metric_from_income_stmt('Tax Rate For Calcs', 0)
        nopat = net_income * (1 - tax_rate)
        invested_capital = self.__get_total_assets() - self.__get_current_liabilities()
        wacc = self.dcf_valuation.get_wacc()
        eva = nopat - (invested_capital * wacc)
        if np.isnan(eva):
            eva = 0
        return eva

    def __calculate_dividend_payout_ratio(self):
        div_yield = self._get_metric_from_df('dividendYield', 0) * 100
        pe_ratio = self._get_metric_from_df('trailingPE', 0)
        if pe_ratio != 0:
            dividend_payout_ratio = div_yield / pe_ratio
        else:
            dividend_payout_ratio = 0
        return dividend_payout_ratio

    def __get_sgr(self):
        roe = self._get_metric_from_df('returnOnEquity', 0)
        dividend_payout_ratio = self.__calculate_dividend_payout_ratio()
        retention_ratio = 1 - dividend_payout_ratio
        sgr = roe * retention_ratio
        return sgr if not pd.isna(sgr) else 0


    def __get_net_current_asset_value_per_share(self):
        current_assets = self.__get_current_assets()
        total_liabilities = self.__get_total_liabilities()
        ordinary_shares_outstanding = self._get_metric_from_balance_sheet('Ordinary Shares Number', 0)
        preferred_shares = self._get_metric_from_balance_sheet('Preferred Shares Number', 0)
        ncavps = self._get_ratio(current_assets - total_liabilities - preferred_shares, ordinary_shares_outstanding)
        return ncavps

    def __get_cap_ex(self):
        return self._get_metric_from_cashflow('Capital Expenditure', 0)

    def __get_ddm_value(self):
        return self.ddm_valuation.get_value()

    def __get_ddm_values(self):
        return self.ddm_valuation.get_valuation_ratios()

    def __get_dcf_value(self):
        return self.dcf_valuation.get_value()

    def __get_dcf_values(self):
        return self.dcf_valuation.get_valuation_ratios()

    def __get_retained_earnings(self):
        retained_earnings_keys = [
            'Retained Earnings',
            'Accumulated Retained Earnings',
            'Undistributed Profits'
        ]
        return self._get_sum_of_keys('Retained Earnings', retained_earnings_keys)

    def __get_current_liabilities(self):
        current_liabilities_keys = [
            'Accounts Payable',
            'Short Term Debt',
            'Other Current Liabilities',
            'Current Deferred Liabilities',
            'Current Deferred Revenue',
            'Current Debt And Capital Lease Obligation',
            'Current Debt',
            'Other Current Borrowings',
            'Commercial Paper',
            'Payables And Accrued Expenses',
            'Payables'
        ]
        return self._get_sum_of_keys('Current Liabilities', current_liabilities_keys)

    def __get_total_liabilities(self):
        total_liabilities_keys = [
            'Total Liabilities Net Minority Interest',
            'Total Non Current Liabilities Net Minority Interest',
            'Other Non Current Liabilities',
            'Trade and Other Payables Non Current',
            'Long Term Debt And Capital Lease Obligation',
            'Long Term Debt',
            'Current Liabilities',
            'Other Current Liabilities',
            'Current Deferred Liabilities',
            'Current Deferred Revenue',
            'Current Debt And Capital Lease Obligation',
            'Current Debt',
            'Other Current Borrowings',
            'Commercial Paper',
            'Payables And Accrued Expenses',
            'Payables',
            'Accounts Payable'
        ]
        return self._get_sum_of_keys('Total Liabilities Net Minority Interest', total_liabilities_keys)

    def __parse_analyst_reccomendations(self):
        try:
            analyst_recommendations = self.stock.recommendations
            strong_buy = analyst_recommendations['strongBuy'].sum()
            buy = analyst_recommendations['buy'].sum()
            hold = analyst_recommendations['hold'].sum()
            sell = analyst_recommendations['sell'].sum()
            strong_sell = analyst_recommendations['strongSell'].sum()
        except KeyError:
            return 0, 0, 0, 0, 0
        return strong_buy, buy, hold, sell, strong_sell

# Assuming DCFValuation and DDMValuation classes are defined elsewhere

# stock = StockData('AAPL')
# data = stock.build_data()
# data.to_csv('dell.csv', index=False)
# print(data)


In [18]:
# Let us build the tickers
df = pd.read_csv('./ticker_test.csv')

tickers = df['Symbol']
print(tickers)

0    AAPL
Name: Symbol, dtype: object


In [19]:
# Stock portfolio class with threading
import yfinance as yf
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

class StockPortfolio:
    def __init__(self, stocks, max_workers=5):
        self.stocks = stocks
        self.max_workers = max_workers
        self.portfolio = self.build_portfolio()
        self.n = len(stocks)

    def build_portfolio(self, save_to_csv=True):
        failed = []
        portfolio = pd.DataFrame()
        print("Building portfolio...")

        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = {executor.submit(self.build_stock_data, stock): stock for stock in self.stocks}

            for future in as_completed(futures):
                stock = futures[future]
                try:
                    stock_data = future.result()
                    if stock_data is not None:
                        portfolio = pd.concat([portfolio, stock_data], ignore_index=True)
                except Exception as e:
                    print(f"Failed to obtain data for {stock}. Error: {e}")
                    failed.append(stock)

        if save_to_csv:
            # Pay attention to output file -> don't overwrite
            portfolio.to_csv('test_data.csv', index=False)
        return portfolio

    def build_stock_data(self, ticker):
        try:
            stock = StockData(ticker)
            print(stock)
            stock_data = stock.build_data()
            return stock_data
        except Exception as e:
            print(f"Failed to obtain data for {ticker}. Error: {e}")
            return None

    def get_portfolio(self):
        return self.portfolio
    
    def __str__(self):
        return f"StockPortfolio({self.stocks})"
    
stock_portfolio = StockPortfolio(tickers)
# print(stock_portfolio.get_portfolio())

Building portfolio...
Stock(AAPL)
Growth Rate: 0
WACC: 0.12134315406777503
Cash Flows: [99584000000.0, 111443000000.0, 92953000000.0, 73365000000.0]
Forecasted FCFs: [73365000000.0, 73365000000.0, 73365000000.0, 73365000000.0]
Discounted FCFs: [65426002498.75495, 58346102405.33073, 52032334788.57466, 46401794669.02937]
Terminal Value: 1002294441438.7327
Discounted Terminal Value: 633929814891.9724
DCF Value: 856136049253.6621
Growth Rate: 0
WACC: 0.12134315406777503
Cash Flows: [99584000000.0, 111443000000.0, 92953000000.0, 73365000000.0]
Forecasted FCFs: [73365000000.0, 73365000000.0, 73365000000.0, 73365000000.0]
Discounted FCFs: [65426002498.75495, 58346102405.33073, 52032334788.57466, 46401794669.02937]
Terminal Value: 1002294441438.7327
Discounted Terminal Value: 633929814891.9724
DCF Value: 856136049253.6621
Failed to obtain data for AAPL. Error: 'Ticker' object has no attribute 'get_history'


In [24]:
# Let's define the stock portfolio class

class StockPortfolio:
    def __init__(self, stocks):
        self.stocks = stocks
        self.n = len(stocks) 
        self.portfolio = self.build_portfolio()


    def build_portfolio(self, save_to_csv=True):
        failed = []
        portfolio = pd.DataFrame()
        print("Building portfolio...")
        for i, stock in enumerate(self.stocks):
            print(f"Building data for {stock}... ({i+1} / {self.n})")  # Correctly using i and self.n
            try:
                stock_data = StockData(stock)
                stock_data_df = stock_data.build_data()
                if stock_data_df.empty:
                    print(f"Failed to obtain data for {stock}.")
                    failed.append(stock)
                else:
                    portfolio = pd.concat([portfolio, stock_data_df], ignore_index=True)
            except Exception as e:
                print(f"Failed to obtain data for {stock}.")
                print(e)
                failed.append(stock)
        
        if save_to_csv:
            portfolio.to_csv('test_data1.csv', index=False)
        
        print(f"Failed to fetch data for: {failed}")
        return portfolio

    def get_portfolio(self):
        return self.portfolio

    def __str__(self):
        return f"StockPortfolio({self.stocks})"


# Run the stock portfolio class
tickers = ["TSLA"]
stock_portfolio = StockPortfolio(tickers)

Building portfolio...
Building data for TSLA... (1 / 1)
Growth Rate: 0
WACC: 0.18198697002284436
Cash Flows: [4357000000.0, 7552000000.0, 3483000000.0, 2701000000.0]
Forecasted FCFs: [2701000000.0, 2701000000.0, 2701000000.0, 2701000000.0]
Discounted FCFs: [2285135173.6540694, 1933299800.7667582, 1635635459.4411416, 1383801599.276115]
Terminal Value: 27435169627.367317
Discounted Terminal Value: 14055842875.513659
DCF Value: 21293714908.65174
Growth Rate: 0
WACC: 0.18198697002284436
Cash Flows: [4357000000.0, 7552000000.0, 3483000000.0, 2701000000.0]
Forecasted FCFs: [2701000000.0, 2701000000.0, 2701000000.0, 2701000000.0]
Discounted FCFs: [2285135173.6540694, 1933299800.7667582, 1635635459.4411416, 1383801599.276115]
Terminal Value: 27435169627.367317
Discounted Terminal Value: 14055842875.513659
DCF Value: 21293714908.65174
Failed to fetch data for: []


In [31]:
# Set up the API request
import requests
api_key = "67deec4f33msh0943ba7741c6d31p1f00f3jsn3e8d30d8d661"
headers = {
    'x-rapidapi-host': "morning-star.p.rapidapi.com",
    'x-rapidapi-key': api_key
}

# Map to performance ID in MorningStar
def map_to_morningstar_id(ticker):
    url = "https://morning-star.p.rapidapi.com/market/v2/auto-complete"
    queryString = {"q": ticker}
    res = requests.get(url, headers=headers, params=queryString).json()
    # print(res['results'])
    performanceId = res['results'][0]['performanceId']
    if performanceId == None:
        performanceId = res['results'][1]['performanceId']
    return performanceId

# Obtain fair value as judged by MorningStar
def get_fair_value(ticker):
    url = "https://morning-star.p.rapidapi.com/stock/v2/get-price-fair-value"
    performanceId = map_to_morningstar_id(ticker)
    # print(performanceId)
    queryString = {"performanceId": performanceId}
    res = requests.get(url, headers=headers, params=queryString).json()["chart"]["chartDatums"]["recent"]["latestFairValue"]
    return res



In [32]:
# Now let's join the value data to the data frame
def join_fair_value(sp500_value):
    for i, ticker in enumerate(sp500_value['Ticker']):
        print(f'Obtaining fair value for {ticker} ({i+1}/{len(sp500_value)})')
        try:
            fair_value = get_fair_value(ticker)
            sp500_value.loc[sp500_value['Ticker'] == ticker, 'Fair Value'] = fair_value
        except Exception as e:
            print(f'Failed to obtain fair value for {ticker}')
            print(e)
sp500_value = pd.read_csv('./portfolio_value_data.csv')
join_fair_value(sp500_value)

# Save the DataFrame to a CSV file
sp500_value.to_csv('./portfolio_value_fair_value_data.csv', index=False)

Obtaining fair value for AAPL (1/4981)
Obtaining fair value for AMZN (2/4981)
Obtaining fair value for GOOG (3/4981)
Obtaining fair value for MSFT (4/4981)
Obtaining fair value for 2222.SR (5/4981)
Failed to obtain fair value for 2222.SR
list index out of range
Obtaining fair value for TSLA (6/4981)
Obtaining fair value for BRK-A (7/4981)
Failed to obtain fair value for BRK-A
list index out of range
Obtaining fair value for NVDA (8/4981)
Obtaining fair value for TSM (9/4981)
Obtaining fair value for UNH (10/4981)
Obtaining fair value for TCEHY (11/4981)
Obtaining fair value for V (12/4981)
Obtaining fair value for WMT (13/4981)
Obtaining fair value for JNJ (14/4981)
Obtaining fair value for JPM (15/4981)
Obtaining fair value for 005930.KS (16/4981)
Failed to obtain fair value for 005930.KS
list index out of range
Obtaining fair value for NSRGY (17/4981)
Obtaining fair value for PG (18/4981)
Obtaining fair value for XOM (19/4981)
Obtaining fair value for RHHBY (20/4981)
Obtaining fair v